<a href="https://colab.research.google.com/github/anzar00/UNet-Models/blob/main/Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNET

In [11]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
def build_unet(input_shape=(584, 565, 3), num_classes=1):
    inputs = keras.Input(shape=input_shape)
    
    # Encoder block
    c1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = keras.layers.MaxPooling2D((2, 2))(c1)
    
    c2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = keras.layers.MaxPooling2D((2, 2))(c2)
    
    c3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = keras.layers.MaxPooling2D((2, 2))(c3)
    
    # Decoder block
    u4 = keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='valid')(p3)
    u4 = keras.layers.concatenate([u4, c2])
    c4 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c4 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    
    u5 = keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='valid')(c4)
    u5 = keras.layers.concatenate([u5, c1])
    c5 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u5)
    c5 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c5)
    
    outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(c5)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
# Build the model
model = build_unet()

ValueError: ignored

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

NameError: ignored

In [ ]:
#Load the DRIVE dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.drive(num_classes=1, load_data=True)

In [ ]:
#Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
#Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
#Plotting the Training and Validation Accuracy 

import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
#Using the Model for Segmentation

# Load a new image from the DRIVE dataset
new_image = X_test[0]
new_image = np.expand_dims(new_image, axis=0)

# Predict the segmentation mask
segmentation_mask = model.predict(new_image)
segmentation_mask = np.squeeze(segmentation_mask)

# Plot the original image and the segmentation mask
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(X_test[0], cmap='gray')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(segmentation_mask, cmap='gray')
plt.title('Segmentation Mask')
plt.show()


In [ ]:
# Load the DRIVE dataset
X = np.load('/content/drive/My Drive/DRIVE/images.npy')
y = np.load('/content/drive/My Drive/DRIVE/masks.npy')

# Preprocess the DRIVE data
X = X / 255.0
y = (y > 0.5).astype(int)

# Build the UNet model
model = build_unet(input_shape=(512, 512, 1), num_classes=1)

# Compile the UNet model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the UNet model
history = model.fit(X, y, epochs=50, validation_split=0.1)

# Evaluate the UNet model
plot_history(history)

# Use the UNet model for segmentation
pred = model.predict(X_test)


Test 1

In [ ]:
# Import required libraries
import os
import numpy as np
import tensorflow as tf
from skimage.io import imread
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation

# Load the DRIVE dataset
def load_dataset(data_path, folder):
    images = []
    masks = []
    
    for image_path, mask_path in zip(sorted(os.listdir(os.path.join(data_path, folder, 'images'))), 
                                      sorted(os.listdir(os.path.join(data_path, folder, 'masks')))):
        images.append(imread(os.path.join(data_path, folder, 'images', image_path)))
        masks.append(imread(os.path.join(data_path, folder, 'masks', mask_path)))
        
    return np.array(images), np.array(masks)

# Load the training dataset
X_train, y_train = load_dataset('/content/drive/My Drive/dataset', 'training')

# Load the test dataset
X_test, y_test = load_dataset('/content/drive/My Drive/dataset', 'test')

# Define the UNet model
def unet(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4


# Check New

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/dataset/

/content/drive/MyDrive/dataset


In [3]:
!pip install tensorflow==2.6.0
!pip install numpy==1.20
!pip install matplotlib
!pip install opencv-python-headless
!pip install imagecodecs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.4/458.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 58.6 MB/s eta 0:00:00
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=c2f2ac2173a9496bcedb57c91d7c9460c43bde331979c11384ab2f4c975af106
  Stored in directory: /root/.cache/pip/wheels/f1/60/77/22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=5e7a9d6e809dd494b4a9f5ed9d82e912f9b97b48e9059f3a23540b1e4afd7674
  Stored in directory: /root/.cache/pip/wheels/

In [4]:
!pip install tensorflow==2.6.0 numpy matplotlib opencv-python-headless imagecodecs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.5-cp38-cp38-manylinux2010_x86_64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.0
    Uninstalling numpy-1.20.0:
      Successfully uninstalled numpy-1.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray-einstats 0.5.1 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jaxlib 0.4.4+cuda11.cudnn82 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jax 0.4.4 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
cmdstanpy 1.1.0 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
bokeh 2.4.3 requires typing-extensions>=3.10.0, but yo

In [ ]:
!ls -l test/mask

total 70
-rw------- 1 root root 3301 Jan 20  2004 01_test_mask.gif
-rw------- 1 root root 3304 Jan 20  2004 02_test_mask.gif
-rw------- 1 root root 3309 Jan 20  2004 03_test_mask.gif
-rw------- 1 root root 3312 Jan 20  2004 04_test_mask.gif
-rw------- 1 root root 3304 Jan 20  2004 05_test_mask.gif
-rw------- 1 root root 3301 Jan 20  2004 06_test_mask.gif
-rw------- 1 root root 3310 Jan 20  2004 07_test_mask.gif
-rw------- 1 root root 3303 Jan 20  2004 08_test_mask.gif
-rw------- 1 root root 3309 Jan 20  2004 09_test_mask.gif
-rw------- 1 root root 3304 Jan 20  2004 10_test_mask.gif
-rw------- 1 root root 3307 Jan 20  2004 11_test_mask.gif
-rw------- 1 root root 3304 Jan 20  2004 12_test_mask.gif
-rw------- 1 root root 3300 Jan 20  2004 13_test_mask.gif
-rw------- 1 root root 3306 Jan 20  2004 14_test_mask.gif
-rw------- 1 root root 3306 Jan 20  2004 15_test_mask.gif
-rw------- 1 root root 3310 Jan 20  2004 16_test_mask.gif
-rw------- 1 root root 3304 Jan 20  2004 17_test_mask.gif
-rw--

In [5]:
!pip install tifffile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tifffile
import imageio
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split

# Define the paths to the training and testing images and masks
train_images_path = './training/images/'
train_masks_path = './training/mask/'
test_images_path = './test/images/'
test_masks_path = './test/mask/'

# Define the image size
IMG_HEIGHT = 256
IMG_WIDTH = 256

# Define the number of channels in the images and masks
IMG_CHANNELS = 3
MASK_CHANNELS = 1

# Load the training images and masks
train_images = sorted(os.listdir(train_images_path))
train_masks = sorted(os.listdir(train_masks_path))

# Create empty arrays for the training images and masks
X_train = np.zeros((len(train_images), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_masks), IMG_HEIGHT, IMG_WIDTH, MASK_CHANNELS), dtype=np.uint8)

# Load the training images and masks into the arrays
for i, image_path in enumerate(train_images):
    img = tifffile.imread(os.path.join(train_images_path, image_path))
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    X_train[i] = img
    
for i, mask_path in enumerate(train_masks):
    mask = imageio.imread(os.path.join(train_masks_path, mask_path))
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
    mask = np.expand_dims(mask, axis=-1)
    Y_train[i] = mask

# Load the testing images and masks
test_images = sorted(os.listdir(test_images_path))
test_masks = sorted(os.listdir(test_masks_path))


# Create empty arrays for the testing images and masks
X_test = np.zeros((len(test_images), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_masks), IMG_HEIGHT, IMG_WIDTH, MASK_CHANNELS), dtype=np.uint8)

# Load the testing images and masks into the arrays
for i, image_path in enumerate(test_images):
    img = tifffile.imread(os.path.join(test_images_path, image_path))
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    X_test[i] = img
    
for i, mask_path in enumerate(test_masks):
    mask = imageio.imread(os.path.join(test_masks_path, mask_path))
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
    mask = np.expand_dims(mask, axis=-1)
    Y_test[i] = mask


In [13]:
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.models import Model

# Define the input shape of the model
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

# Define the number of filters in the first convolutional layer
n_filters = 32

# Define the U-Net model
inputs = Input(input_shape)

# Contracting path
c1 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)
p1 = Dropout(0.1)(p1)

c2 = Conv2D(n_filters*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = Conv2D(n_filters*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)
p2 = Dropout(0.1)(p2)

c3 = Conv2D(n_filters*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = Conv2D(n_filters*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)
p3 = Dropout(0.2)(p3)

c4 = Conv2D(n_filters*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = Conv2D(n_filters*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = MaxPooling2D((2, 2))(c4)
p4 = Dropout(0.2)(p4)

c5 = Conv2D(n_filters*16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = Conv2D(n_filters*16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
p5 = MaxPooling2D((2, 2))(c5)
p5 = Dropout(0.3)(p5)

# Expansive path
u6 = UpSampling2D((2, 2))(p5)
c6 = Conv2D(n_filters*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = concatenate([c6, c5])
c6 = Conv2D(n_filters*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
c6 = Conv2D(n_filters*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
c6 = Dropout(0.2)(c6)

u7 = UpSampling2D((2, 2))(c6)
c7 = Conv2D(n_filters*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = concatenate([c7, c4])
c7 = Conv2D(n_filters*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
c7 = Conv2D(n_filters*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
c7 = Dropout(0.2)(c7)

u8 = UpSampling2D((2, 2))(c7)
c8 = Conv2D(n_filters*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = concatenate([c8, c3])
c8 = Conv2D(n_filters*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
c8 = Conv2D(n_filters*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
c8 = Dropout(0.1)(c8)

u9 = UpSampling2D((2, 2))(c8)
c9 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = concatenate([c9, c2])
c9 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
c9 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
c9 = Dropout(0.1)(c9)

u10 = UpSampling2D((2, 2))(c9)
c10 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u10)
c10 = concatenate([c10, c1])
c10 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)
c10 = Conv2D(n_filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)

outputs = Conv2D(1, (1, 1), activation='sigmoid')(c10)

model = Model(inputs=[inputs], outputs=[outputs])


AttributeError: ignored

In [ ]:
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
pwd: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the batch size and number of epochs
BATCH_SIZE = 4
EPOCHS = 50

# Define the training and validation data generators
train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_data_gen.flow_from_directory(
    './training/',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed=42,
    shuffle=True
)

val_generator = val_data_gen.flow_from_directory(
    './test/',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed=42,
    shuffle=True
)

# Define the model checkpoint
checkpoint_path = "./checkpoint/model.ckpt"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)


# Compile the model
from tensorflow.keras.optimizers import Adam

# Set the learning rate and optimizer
lr = 1e-4
optimizer = Adam(lr=lr)

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[checkpoint_callback]
)


AttributeError: ignored

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(train_generator, steps_per_epoch=len(train_image_paths)//batch_size, epochs=epochs, 
                    validation_data=val_generator, validation_steps=len(val_image_paths)//batch_size)

# save the trained model
model.save('unet_model.h5')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_58 (Conv2D)             (None, 256, 256, 32  896         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_59 (Conv2D)             (None, 256, 256, 32  9248        ['conv2d_58[0][0]']              
                                )                                                           